In [1]:
from multidimvis_main import *

In [2]:
# Human 
G = nx.read_edgelist('input/ppi_elist.txt',data=False)

In [ ]:
# Yeast 
data = pickle.load( open( "input/BIOGRID-ORGANISM-Saccharomyces_cerevisiae_S288c-3.5.185.mitab.pickle", "rb" ) )

Counter(data['Interaction Detection Method'])
Counter(data['Interaction Types'])

filter_score = data[
                    #(data['Interaction Types'] == 'psi-mi:"MI:0915"(physical association)') +
                    (data['Interaction Types'] == 'psi-mi:"MI:0407"(direct interaction)') 
                    #&
                    #(data['Taxid Interactor A'] == "taxid:559292") & 
                    #(data['Taxid Interactor B'] == "taxid:559292") 
]

g = nx.from_pandas_edgelist(filter_score, '#ID Interactor A', 'ID Interactor B')
g.remove_edges_from(nx.selfloop_edges(g)) #remove self loop

G_cere = g.subgraph(max(nx.connected_components(g), key=len)) # largest connected component (lcc)

# ESSENTIAL GENES 
cere_gene =pd.read_csv("input/Saccharomyces cerevisiae.csv",
           delimiter= ',',
           skipinitialspace=True)
essential_cere = cere_gene[(cere_gene['essentiality status'] == 'E')]
essential_genes_cere_list =  essential_cere['symbols'].tolist()

degree= dict(G_cere.degree())

mg = mygene.MyGeneInfo()
a = mg.querymany(essential_genes_cere_list, scopes='symbol', species=559292)
essential_genes_cere_names = pd.DataFrame.from_dict(a)
essential_genes_cere_entrez =  essential_genes_cere_names['entrezgene'].tolist()

cleaned_entrez_list = [x for x in essential_genes_cere_entrez if str(x) != 'nan']

degree_formatted={}
for k, v in degree.items():
    degree_formatted[k.replace("entrez gene/locuslink:","")] = v
    
index= []
essential = []
for i in cleaned_entrez_list:
    for (key, val) in degree_formatted.items():
        if i==key:
            index.append(key)
            essential.append(val)  

no_essential_cere = cere_gene[(cere_gene['essentiality status'] == 'NE')]
no_essential_genes_cere_list =  no_essential_cere['symbols'].tolist()
b = mg.querymany(no_essential_genes_cere_list, scopes='symbol', species=559292)
no_essential_genes_cere_names = pd.DataFrame.from_dict(b)
no_essential_genes_cere_entrez =  no_essential_genes_cere_names['entrezgene'].tolist()
cleaned_entrez_list_no = [x for x in no_essential_genes_cere_entrez if str(x) != 'nan']

index= []
no_essential = []
for i in cleaned_entrez_list_no:
    for (key, val) in degree_formatted.items():
        if i==key:
            index.append(key)
            no_essential.append(val)
            
#df_cere = pd.DataFrame({'essential': pd.Series(essential), 'no_essential': pd.Series(no_essential)})


no_ess_id = no_essential_genes_cere_names['entrezgene']
ess_id = essential_genes_cere_names['entrezgene']
G = G_cere

#edge_list = nx.write_edgelist(G, "Yeast_edgelist_directinteractiononly.txt")

### GRAPH BASED MATRICES

#### Adjacency

In [3]:
%%time

A = nx.adjacency_matrix(G)
DM_adj = A.toarray()

CPU times: user 1 s, sys: 181 ms, total: 1.18 s
Wall time: 1.18 s


In [4]:
df_adj = pd.DataFrame(DM_adj, columns = list(G.nodes()), index=list(G.nodes()))

In [5]:
df_adj.to_csv(r'output_csv/Adjacency_Dataframe_Human.csv', index = True)

#### Shortest Path Length

In [ ]:
%%time

d_idx_entz = {}
cc = 0
for entz in sorted(G.nodes()):
    d_idx_entz[cc] = entz
    cc += 1

Mspl = np.zeros(len(list(G.nodes())))

for n1 in range(len(list(G.nodes()))):
    vec = []
    for n2 in range(len(list(G.nodes()))):
        geneA = d_idx_entz[n1]
        geneB = d_idx_entz[n2]
        try:
            spl = nx.shortest_path_length(G,geneA,geneB)
            vec.append(spl)
        except nx.NetworkXNoPath:
            print('no path')
        
    Mspl = np.vstack((Mspl,vec))
Mspl = np.delete(Mspl, (0), axis=0)

DM_spl = Mspl 

In [ ]:
df_spl = pd.DataFrame(DM_spl, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_spl.to_csv(r'output_csv/SPL_Dataframe_Human.csv', index = True)

#### Markov: Random Walk with restart probability

In [4]:
%%time

# Restart probability
r = .8 # originally 0.8
alpha = 1.0 # indicating "randomness" 

DM_m = rnd_walk_matrix2(A, r, alpha, len(G.nodes()))

DM_m_df = pd.DataFrame(DM_m)
# DM_m_df.sum(axis=0)

DM_m_transposed = DM_m_df.T
DM_m_transposed.index = list(G.nodes())

CPU times: user 6min 15s, sys: 12.6 s, total: 6min 28s
Wall time: 1min 53s


In [5]:
%%time 

metric = "cosine" 
DM_m_new = pd.DataFrame(distance.squareform(distance.pdist(DM_m_transposed, metric)))

CPU times: user 46min 38s, sys: 24.6 s, total: 47min 3s
Wall time: 47min 32s


In [ ]:
df_m = pd.DataFrame(DM_m_new, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_m.to_csv(r'output_csv/RWR_'+metric+'_Dataframe_Human.csv', index = True)

#### -log Markov

In [ ]:
# MARKOV / RWR - LOG

#min_log = lambda t: -np.log(t)
#DM_mlog = np.array([min_log(x/max(x)) for x in DM_m])

In [ ]:
df_mlog = pd.DataFrame(DM_mlog, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_mlog.to_csv(r'output_csv/RWR_log_'+metric+'_Dataframe_Human.csv', index = True)